<a href="https://colab.research.google.com/github/Disha-Sikka/FineTune_and_GradCam/blob/main/FineTune_%26_GradCam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!kaggle datasets download -d puneet6060/intel-image-classification
!unzip -q intel-image-classification.zip


Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
 96% 334M/346M [00:05<00:00, 42.8MB/s]
100% 346M/346M [00:05<00:00, 62.6MB/s]


In [13]:
from torchvision import datasets, transforms
from torch.utils.data import dataloader

In [16]:
transform= transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])